In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import cv2

In [2]:
def int_coords(coord_old,coord_new):
    
    if np.sum(coord_old == coord_new)==coord_old.size:
        return [(0,0),(0,0)]
    else:
        print('Coords old:',np.sum(coord_old==coord_new),len(coord_old),coord_old)
    height_old = coord_old[1][1]-coord_old[0][1]
    width_old = coord_old[1][0]-coord_old[0][0]
    height_new = coord_new[1][1]-coord_new[0][1]
    width_new = coord_new[1][0]-coord_new[0][0]
    if width_new>width_old:
        if (coord_new[1][0])>(coord_old[1][0]):
            new_left_x = coord_old[1][0]
            new_right_x = coord_new[1][0]
            new_left_y = coord_new[0][1]
            new_right_y = coord_new[1][1]
#             print('old',coord_old,'new',coord_new,'right',[(new_left_x,new_left_y),(new_right_x,new_right_y)])
            
        else:
            new_left_x = coord_old[0][0]
            new_right_x = coord_new[0][0]
            new_left_y = coord_new[0][1]
            new_right_y = coord_new[0][1]+height_new
#             print('old',coord_old,'new',coord_new,'left',[(new_left_x,new_left_y),(new_right_x,new_right_y)])
#     elif height_new>height_old:
#         if (coord_new[1][1])>(coord_old[1][1]):
#             new_left_x = coord_old[1][0]
#             new_right_x = coord_new[1][0]
#             new_left_y = coord_old[1][1]-width_old
#             new_right_y = coord_new[1][1]
#         else:
#             new_left_x = coord_new[0][0]
#             new_right_x = coord_old[0][0]+width_old
#             new_left_y = coord_new[0][1]
#             new_right_y = coord_old[0][1]
    else:
        new_left_x = coord_old[0][0]
        new_right_x = coord_old[1][0]
        new_left_y = coord_old[0][1]
        new_right_y = coord_old[1][1]
    return [(new_left_x,new_left_y),(new_right_x,new_right_y)]

In [3]:
import string
# For one char
def maps(words_old,words_new,coords_old,coords_new,dir,gt,id,coords_all,coords_new_all):
    '''
    words_old = list of words matched which are the variations of original word
    words_new = list of words matched after extending the region to include 1 more char
    coords_old = region coordinates in a list of list of tuples of (top left, bottom right) before extending the region
    coords_new = region after extending to include one char
    returns:
    c_map = dictionary of characters to coordinates
    '''
    c_map = {}
    g_map = {}
    for a in string.ascii_lowercase:
        c_map[a] = []
    for A in string.ascii_uppercase:
        c_map[A] = []
    for i,word in enumerate(words_old):  
        if dir[i]:
            if word == words_new[i][1:]:
                char_coords = int_coords(coords_old[i],coords_new[i])
                l = c_map.get(words_new[i][0],[])
                l.append(char_coords)
# #                 print('left',char_coords,'old',coords_all[i],'new',coords_new_all[i],words_new[i][0],'gt',gt[i],'old',words_old[i],'new',words_new[i],'id',id[i],'dir',dir[i])
                c_map[words_new[i][0]] = l
            elif word == words_new[i][:-1]:
                char_coords = int_coords(coords_old[i],coords_new[i])
                l = c_map.get(words_new[i][-1],[])
                l.append(char_coords)
#                 print('left',char_coords,'old',coords_all[i],'new',coords_new_all[i],words_new[i][-1],'gt',gt[i],'old',words_old[i],'new',words_new[i],'id',id[i],'dir',dir[i])
                c_map[words_new[i][-1]] = l
    return c_map

In [4]:
coords_old_all = np.load('../detection_outputs/D0090-5242001.npy')
coords_new_all = np.load('../detection_outputs_new/D0090-5242001.npy')
coords_new = np.array([[coords_new_all[i][3],coords_new_all[i][1]] for i in range(coords_new_all.shape[0])])
ids = np.load('../images_to_extend/image_dir_D0090-5242001.npy')
coords_old = np.array([[coords_old_all[i][3],coords_old_all[i][1]] for i in ids[0]])
print(coords_old.shape,coords_new.shape,ids.shape)
ids

((469, 2, 2), (469, 2, 2), (3, 469))


array([[  0,   1,   2, ..., 531, 532, 533],
       [  0,   0,   0, ...,   1,  -1,   0],
       [  5,   5,   4, ...,   6,   3,   3]])

In [5]:
import pandas as pd
import ast
# csv = pd.read_csv('../data/D0090-5242001.csv',names=['ids','dir','old','new','gt'])
csv = pd.read_csv('/media/d/DATA/NN/torch-phoc/mudit/torch-phoc/extension_info/D0090-5242001.csv',names=['ids','dir','old','new','gt'])
words_old = [ast.literal_eval(word)[0] for word in csv['old']]
words_new = [ast.literal_eval(word)[0] for word in csv['new']]
dir = list(csv['dir'])
gt = list(csv['gt'])
# gt = [ast.literal_eval(word)[0] for word in csv['gt']]

In [6]:
# Sample data
# words_old = ['wor','ord']
# words_new = ['word','word']
# coords_old = [[(1000,2000),(2000,3000)],[(2000,2400),(2400,2500)]]
# coords_new = [[(1000,2000),(2500,3000)],[(2000,2000),(2400,2500)]]
d = maps(words_old,words_new,coords_old,coords_new,dir,gt,ids[0],coords_old_all,coords_new_all)

('Coords old:', 3, 2, array([[1429, 2520],
       [1540, 2557]]))
('Coords old:', 3, 2, array([[1296, 1664],
       [1412, 1682]]))
('Coords old:', 3, 2, array([[3738, 1643],
       [3837, 1670]]))
('Coords old:', 3, 2, array([[1135,  875],
       [1271,  907]]))
('Coords old:', 3, 2, array([[2277, 1602],
       [2394, 1634]]))
('Coords old:', 3, 2, array([[3738, 2463],
       [3844, 2491]]))


In [7]:
print(d)

{'.': [[(3844, 2463), (3865, 2491)]], 'A': [], 'C': [], 'B': [], 'E': [[(2394, 1602), (2417, 1634)]], 'D': [], 'G': [], 'F': [], 'I': [], 'H': [], 'K': [], 'J': [], 'M': [], 'L': [], 'O': [], 'N': [], 'Q': [], 'P': [], 'S': [[(1296, 1664), (1258, 1682)]], 'R': [], 'U': [], 'T': [[(1429, 2520), (1402, 2557)]], 'W': [], 'V': [], 'Y': [], 'X': [], 'Z': [], 'a': [], 'c': [], 'b': [], 'e': [], 'd': [], 'g': [], 'f': [], 'i': [], 'h': [], 'k': [], 'j': [], 'm': [], 'l': [], 'o': [], 'n': [[(0, 0), (0, 0)], [(1271, 875), (1288, 907)]], 'q': [], 'p': [], 's': [], 'r': [], 'u': [], 't': [], 'w': [], 'v': [], 'y': [[(3837, 1643), (3853, 1670)]], 'x': [], 'z': []}


In [8]:
def plot_char(char,c_map,map_name,img=None):
    if type(img)!=np.ndarray:
        im = Image.open(map_name)
        img = np.array(im)
    coords = c_map[char]
    for coord in coords:
#             print(coords)
#             print(coord[0],'0',coord[1])
            cv2.rectangle(img,coord[0],coord[1],(255,0,0),5)
#     If each character has to be saved:
#     im = Image.fromarray(img)
#     im.save('h_'+'D0090-5242001.tiff')
    return img

In [9]:
print(d)
map_name = '../data/maps/D0090-5242001.tiff'
im = Image.open(map_name)
img = np.array(im)
print(type(img))
for c in d.keys():
    img =  plot_char(c,d,'../data/maps/D0090-5242001.tiff',img)
im = Image.fromarray(img)
im.save('chars_'+'D0090-5242001.tiff')

# f = plt.figure()
# f.savefig('d_'+'D0090-5242001.eps',format='eps',dpi=300)
# f

{'.': [[(3844, 2463), (3865, 2491)]], 'A': [], 'C': [], 'B': [], 'E': [[(2394, 1602), (2417, 1634)]], 'D': [], 'G': [], 'F': [], 'I': [], 'H': [], 'K': [], 'J': [], 'M': [], 'L': [], 'O': [], 'N': [], 'Q': [], 'P': [], 'S': [[(1296, 1664), (1258, 1682)]], 'R': [], 'U': [], 'T': [[(1429, 2520), (1402, 2557)]], 'W': [], 'V': [], 'Y': [], 'X': [], 'Z': [], 'a': [], 'c': [], 'b': [], 'e': [], 'd': [], 'g': [], 'f': [], 'i': [], 'h': [], 'k': [], 'j': [], 'm': [], 'l': [], 'o': [], 'n': [[(0, 0), (0, 0)], [(1271, 875), (1288, 907)]], 'q': [], 'p': [], 's': [], 'r': [], 'u': [], 't': [], 'w': [], 'v': [], 'y': [[(3837, 1643), (3853, 1670)]], 'x': [], 'z': []}
<type 'numpy.ndarray'>


In [10]:
for i in range(coords_old.shape[0]):
    d[words_old[i]]=[[(coords_old[i][0][0],coords_old[i][0][1]),(coords_old[i][1][0],coords_old[i][1][1])]]
for c in d.keys():
    img =  plot_char(c,d,'../data/maps/D0090-5242001.tiff',img)
im = Image.fromarray(img)
im.save('all_'+'D0090-5242001.tiff')

In [11]:
d = np.array(d)
np.save('dict.npy',d)